# Сбор сессии вопросов и ответов из стенограмм ГосДумы

Цель: получить обсуждение законопроектов в первом чтении по возможности для каждого комитета

Источник: http://transcript.duma.gov.ru/

## Алгоритм:

1) Вручную указать ссылки на страницы со списком хроникам для каждого созыва (страницы, на которых собраны ссылки).

   Переменная: **session_X_chronicles_pages**
   
2) Вручную указать ссылки на страницы со списком стенограмм для каждого созыва.

   Переменная: **session_X_stenograms_pages**

3) Получить ссылки на все хроники созыва X
   
   Функция: **get_all_chronicles_links**(session_chronicles_pages)   

   Полученная  с помощью нее переменная: **session_X_chronicles_links**
   
4) Получить ссылки на все стенограммы созыва X
   
   Функция: **get_all_stenograms_links**(session_stenograms_pages)   

   Полученная  с помощью нее переменная: **session_X_stenograms_links** 
   
5) Из каждой хроники получить список пунктов повестки, в которых законопроекты первого чтения имели обсуждение (были заданы вопросы). 

   Функция для одной хроники: **get_discussed_numbers**
    
   Функция для списка ссылок на хроники: **get_all_discussed_numbers**(session_chronicles_links)
    
   Полученная  с помощью нее переменная: **session_X_discussed_numbers**
   
6) Разделить список обсужденных вопросов на коммитеты.

   Функция: **get_variables_links_by_commities**(session_discussed_numbers, n)
   
   Полученная  с помощью нее переменная: **session_8_dialogies_by_commities**
   (хранит список на str названия переменных для комитетов)
   

7) Из списка обсужденных вопросов получить сессию ответов и вопросов.

   Вспомогательная функция **get_clean_transcript** по ссылке на стенограмму получает транскрипт 
    всего заседания в виде [кто говорит][что говорит]
    
   Вспомогательная функция **get_certain_discussion**(url, number, additional_speech) получает список формата 
    [вопрос, ответ, вопрос, ответ..... ] - сессию ответов и вопросов по данной стенограмме.
    
   Функция **get_all_clean_dialogues_by_commitee**(session_dialog_commitee, session_stenograms_links_dict) вернет для списка всех обсужденных вопросов по каждому коммитету список [вопрос, ответ, вопрос, ответ..... ]
    

## Получение данных устроено следующим образом:

### Алгоритм:

8, и любая другая цифра, изменяется и указывает на номер Собрания

    1) session_8_stenograms_pages = [внести]
    2) session_8_chronicles_pages = [внести]
    3) session_8_chronicles_links = get_all_chronicles_links(session_8_chronicles_pages)
    4) session_8_stenograms_links_dict = get_all_stenograms_links(session_8_stenograms_pages)
    5) session_8_discussed_numbers = get_all_discussed_numbers(session_8_chronicles_links)
    6) Создание session_8_ready_dialogs 
    7) Выгрузка файлов

In [82]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
from transliterate import translit

In [83]:
list_of_commities = ['делам общественных объединений и религиозных организаций',
 'конституционному законодательству и государственному строительству',
 'финансовому рынку',
 'контролю',
 'просвещению',
 'промышленности и торговле',
 'делам СНГ и связям с соотечественниками',
 'природным ресурсам, природопользованию и экологии',
 'Регламенту и организации работы Государственной думы',
 'культуре',
 'экономической политике',
 'гражданскому, уголовному, арбитражному и процессуальному законодательству',
 'бюджету и налогам',
 'безопасности и противодействию коррупции',
 'охране здоровья',
 'транспорту и развитию транспортной инфраструктуры',
 'обороне',
 'вопросам собственности, земельным и имущественным отношениям',
 'развитию Дальнего Востока и Арктики',
 'промышленности',
 'образованию',
 'малому и среднему предпринимательству',
 'науке и высшему образованию',
 'транспорту',
 'туризму и развитию туристической инфраструктуры',
 'федеративному устройству и вопросам местного самоуправления',
 'развитию гражданского общества, вопросам общественных и религиозных объединений',
 'делам национальностей',
 'защите конкуренции',
 'региональной политике и проблемам Севера и Дальнего Востока',
 'вопросам семьи, женщин и детей',
 'региональной политике и местному самоуправлению',
 'земельным отношениям и строительству',
 'экономической политике, инновационному развитию и предпринимательству',
 'делам Содружества Независимых Государств, евразийской интеграции и связям с соотечественниками',
 'жилищной политике и жилищно-коммунальному хозяйству',
 'физической культуре, спорту и делам молодёжи',
 'строительству и жилищно-коммунальному хозяйству',
 'энергетике',
 'вопросам собственности',
 'международным делам',
 'экологии, природным ресурсам и охране окружающей среды',
 'молодежной политике',
 'государственному строительству и законодательству',
 'информационной политике, информационным технологиям и связи',
 'труду, социальной политике и делам ветеранов',
 'науке и наукоёмким отношениям',
 'физической культуре и спорту',
 'аграрным вопросам']

In [84]:
def get_links_chronicles(url):
    '''
    Function will return a list in which 
    every element shows
    [date of session][link to chronicle of session]
    '''
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")

    raw_chronicles = []
    for link in soup.find_all("a", href = re.compile(r"^/node/\d+/")):
        raw_chronicles.append(str(link))    

    links_chronicles = []
    beginning = 'http://transcript.duma.gov.ru/'
    for i in raw_chronicles:
        link_start = i.find('node')
        link = beginning +  i[link_start:20]
        date_start = i.find('Хроника')
        date_ending = i.find('<', i.find('Хроника'))
        date = i[date_start:date_ending]
        links_chronicles.append([date, link])

    return links_chronicles

In [85]:
def get_links_stenograms(url):
    
    '''    
    На вход: ссылка на страницу со ссылками на стенограмму.
    
    Возвращает: список формата    
    [дата сессии][ссылка на стенограмму]
    
    '''
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")

    raw_stenograms = []
    for link in soup.find_all("a", href = re.compile(r"^/node/\d+/")):
        raw_stenograms.append(str(link))    

    links_stenograms = []
    beginning = 'http://transcript.duma.gov.ru/'
    for i in raw_stenograms:
        link_start = i.find('node')
        link = beginning +  i[link_start:20]
        date_start = i.find('Стенограмма')
        date_ending = i.find('<', i.find('Стенограмма'))
        date = i[date_start:date_ending]
        links_stenograms.append([date, link])

    return links_stenograms

In [86]:
def get_all_chronicles_links(session_chronicles_pages):    
    
    '''
    На вход: страницы, на которых много ссылкок на хроники.
    Возвращает: список всех ссылок на все эти хроники. 
    '''
    
    session_chronicles_links = []
    for i in session_chronicles_pages:
        chron = get_links_chronicles(i)
        only_links = list(map(lambda x: x[1], chron))
        session_chronicles_links += (only_links)
        
    return session_chronicles_links

In [87]:
def get_all_stenograms_links(session_stenograms_pages):
    
    '''
    На вход: страницы, на которых много ссылкок на стенограммы.
    Возвращает: словарь всех ссылок на все эти стенограммы
    (ключ - дата)
    '''
    
    session_stenograms_links = []

    for i in session_stenograms_pages:
        session_stenograms_links.extend(get_links_stenograms(i))
        
    
    # создадим словарь, чтобы было удобнее обращаться к элементам

    session_stenograms_links_dict = dict()
    for i in session_stenograms_links:
        key = i[0]
        key = key[key.find(' ', key.find('засед')) +1 :]
        key = key.replace('.', '')
        value = i[1]
        session_stenograms_links_dict[key] = value
        
    return session_stenograms_links_dict

In [88]:
def get_legit_answer_question_structure(dialog):
        
    for i in range(0, len(dialog)):
        if i % 2 == 0:
            if '?' not in dialog[i]:
                dialog.remove(dialog[i])
                return get_legit_answer_question_structure(dialog)
            if '?' in dialog[i]:
                continue
        if i % 2 == 1: 
            if '?' not in dialog[i]:
                continue
            if '?' in dialog[i]:
                dialog.remove(dialog[i])
                return get_legit_answer_question_structure(dialog)
                    
    return dialog

In [89]:
def get_discussed_numbers(url):
    
    '''
    на вход - ссылка на хронику.
    returns:
    * дата заседания
    * пункт повестки, в котором было обсуждение,
    * индикатор того, был ли содоклад,
    * какой комитет выступал    
    
    '''

    #url = 'http://transcript.duma.gov.ru/node/5969/'

    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")
    date_start = soup.text.find('Хроника заседания')
    date_end = soup.text.find('.')
    date = soup.text[date_start: date_end+1]    

    start_phrase = 'депутаты приступили к рассмотрению законопроектов в первом чтении'
    if start_phrase is None:
        return None
    start_tag = soup.find(lambda tag: tag.name == 'p' and start_phrase in tag.text)
    if start_tag is None:
        return None
    start_index = soup.find_all('p').index(start_tag)

    results = []

    for p in soup.find_all('p')[start_index+1:]:
        phrases = ['ответил на вопросы', 'ответила на вопросы', \
                   'ответили на вопросы', 'на вопросы ответил', \
                   'на вопросы ответила', 'на вопросы ответили']
        if any(phrase in p.text.lower() for phrase in phrases):
            item = []
            item.append(date)
            paragraph = p.text.lower()
            start = paragraph.find('повестки дня')
            #number = paragraph[start-3: start]
            helper_to_find_number = paragraph[start-9: start]
            number = ''
            for el in helper_to_find_number:
                if el.isdigit():
                    number += el
                if el == '.':
                    number = 'no'
                    break
                if el == 'и':
                    break
            item.append(number)
            
            other_speaker = 0
            if 'содоклад' in paragraph:
                other_speaker = 1
            if 'содоклад' not in paragraph:
                if 'выступил' in paragraph and 'изложил' in paragraph:
                    other_speaker = 1
                if 'выступил' in paragraph and paragraph.count('выступил') > 1:
                    other_speaker = 1
                    
            item.append(other_speaker)
            if 'комитета государственной думы по' in paragraph:
                found_options = ['']
                # устранение ошибки по типу: "культуре", "культуре и спорту"
                # возьмем самое длинное (=самое полное) совпадение
                s = 0 # нет комитете
                for c in list_of_commities:
                    if c in paragraph: 
                        s += 1
                        found_options.append(c)
                if s == 0: # комитет не нашелся в списке
                    found_options.append('other')
                
                item.append(max(found_options))
                        
            if 'комитета государственной думы по' not in paragraph:
                item.append('zero')
                
            results.append(item)
            
    results = pd.DataFrame(results)
    results.columns = ['дата', 'пункт повестки', 'наличие содоклада', 'комитет']
    results = results[results['пункт повестки'] != 'no']
            
    return results

In [90]:
def get_all_discussed_numbers(session_chronicles_links):
    
    '''
    На вход: ссылки на все хроники
    На выход: список указаний на то, в какую дату какой пункт повестки
    имел обсуждение (были заданы вопросы), был ли представлен содоклад,
    к какому комитету относится обсуждение
    
    '''
    
    session_discussed_numbers = []
    
    for i in session_chronicles_links:
        session_discussed_numbers.append(get_discussed_numbers(i))
        
    session_discussed_numbers = [i for i in session_discussed_numbers if i is not None]
    
    session_discussed_numbers = pd.concat(session_discussed_numbers)
        
    return session_discussed_numbers    

In [91]:
# словарь с соответствие названия коммитета и транслита первых пяти букв

dict_translits = dict()
for i in list_of_commities:
    translited = translit(i, 'ru', reversed=True)
    if 'voprosam' in translited and 'agra' not in translited:
        translited = translited[translited.find(' ') + 1:]
    translited = translited[:5]
    translited = translited.replace("'", '')
    dict_translits[i] = translited
dict_translits

{'делам общественных объединений и религиозных организаций': 'delam',
 'конституционному законодательству и государственному строительству': 'konst',
 'финансовому рынку': 'finan',
 'контролю': 'kontr',
 'просвещению': 'prosv',
 'промышленности и торговле': 'promy',
 'делам СНГ и связям с соотечественниками': 'delam',
 'природным ресурсам, природопользованию и экологии': 'priro',
 'Регламенту и организации работы Государственной думы': 'Regla',
 'культуре': 'kult',
 'экономической политике': 'ekono',
 'гражданскому, уголовному, арбитражному и процессуальному законодательству': 'grazh',
 'бюджету и налогам': 'bjudz',
 'безопасности и противодействию коррупции': 'bezop',
 'охране здоровья': 'ohran',
 'транспорту и развитию транспортной инфраструктуры': 'trans',
 'обороне': 'oboro',
 'вопросам собственности, земельным и имущественным отношениям': 'sobst',
 'развитию Дальнего Востока и Арктики': 'razvi',
 'промышленности': 'promy',
 'образованию': 'obraz',
 'малому и среднему предпринимате

In [92]:
def get_variables_links_by_commities(session_discussed_numbers, n):
    
    '''
    Функция создает переменные со всеми датами обсужденных вопросов
    для каждого комитета отдельно
    
    Возвращает список из созданных переменных
    '''    
    session_dialogies_by_commities = []
    
    
    n = str(n)
    for i in set(session_discussed_numbers['комитет']):
        if i not in [None]:
            
            translited = translit(i, 'ru', reversed=True)
            if 'voprosam' in translited and 'agra' not in translited:
                translited = translited[translited.find(' ') + 1:]
            translited = translited[:5]
            name = 'session_' + n + '_dialog_commitee_'+ str(translited)
            name = name.replace("'", '')
            globals()[name] = session_discussed_numbers[session_discussed_numbers['комитет'] == i]
            session_dialogies_by_commities.append(name)
        if i == None:
            continue  
             
    return session_dialogies_by_commities

In [93]:
def get_clean_transcript(url):
    
    '''
    На вход: ссылка на стенограмму
    На выход: транскрипт стенограммы в формате 
    [кто говорит, что говорит]
    
    '''
    
    #url = "http://transcript.duma.gov.ru/node/5750/"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")
    try:
        soup.i.decompose()
    # здесь возможно нужен счетчик ошибок, чтобы не было stackoverflow error 
    except AttributeError as e:
        print('I encountered website error, rerunning the function')
        get_clean_transcript(url)
        
    discussion = []

    who = ''
    what = ''
    for i in soup.find_all(['p']):
        if i.find(['b']):
            if len(who) > 0:
                discussion.append([who, what])
            who = (i.find(['b']).text).replace(',', '')
            what = (i.text).replace(i.find(['b']).text, '')
            if 'фракц' in what or 'представитель' in what.lower() \
            or 'председатель' in what.lower():
                what = ''
        else:
            what += i.text

    return discussion

In [94]:
def get_certain_discussion(url, number, additional_speech):
    
    '''
    Функция возвращает сессию вопросов и ответов по определенному пункту повестки дня
    На вход: ссылка, номер повестки, индикатор наличия содоклада
    На выход: 
        список: за вопросом следует ответ
    
    '''
    number = str(number)
    #url = "http://transcript.duma.gov.ru/node/5750/"
    corpus = get_clean_transcript(url)
    
    markers = [number + '-й вопрос',
    'пункт ' + number,
    number + '-й пункт',
    'рассмотрению вопроса ' + number,
    number + '-й, проект',
    number + '-го вопроса',
    number + ' вопроса',
    number + '-му вопросу',
    'вопрос ' + number,
    number + ' вопрос',
    'пункту ' + number ]

    
    number_mentions = []
    
    # найти начало обсуждения определенного пункта
    for index, value in enumerate(corpus):
        if any(marker in (value[1]).lower() for marker in markers)\
            and value[0] == 'Председательствующий.': # только он объявляет повестку
                number_mentions.append(index)
                #просьба рассмотреть вне очереди
        if any(marker in (value[1]).lower() for marker in markers) \
            and ('просьб' in value[1] or 'отклон' in value[1])\
            and value[0] != 'Председательствующий.':
            number_mentions.append(index+1)
        if 'снять' in value[1]:
            return []
                
        
    if len(number_mentions) > 0:
        start = number_mentions[0]
    if len(number_mentions) == 0: # ведущий не говорил четко о начале конкретного пункта
        return []
    
    end = -1
    # найти конец обсуждения
    for i in range(start+1, len(corpus)):
        if 'желающие выступить' in corpus[i][1].lower():
            end = i
            break
        if "результаты голосования" in corpus[i][1].lower():
            end = i
            break
        if "покажите результаты" in corpus[i][1].lower():
            end = i
            break
            
    if end == -1:
        return []
    
    # вырезать диалог без дополнительных комментариев
    dialog = corpus[start:end]
    dialog[:] = [i for i in dialog if i[0] != 'Председательствующий.']
    dialog[:] = [i for i in dialog if i[0] != 'Из зала.']
    
    # склеить реплики одного и того же человека (могли быть прерваны председателем)
    for i in range(1, len(dialog)):
        if dialog[i][0] == dialog[i-1][0]:
            name  = dialog[i][0]
            content = dialog[i-1][1] + dialog[i][1]
            dialog[i-1] = [] # заменим пустым списком, чтобы не изменять длину диалога
            dialog[i] = [name, content]
            
    # убрать пустые элементы
    dialog[:] = [i for i in dialog if len(i) > 0]
    
    
    person1 = dialog[0][0]
    if additional_speech == 1:
        try:
            person2 = dialog[1][0]
        except IndexError as ie:
            return []
    
    # убрать выступление докладчика и содокладчика, интересуют только вопросы и ответы
    
    dialog[:] = dialog[1:]
    if additional_speech == 1:   
        dialog[:] = dialog[1:]
        
        speakers = [person1, person2]
        
        # склеить ответы докладчика и содокладчика, если они рядом 
        # (отвечают на один и тот же вопрос)
        
        for i in range(1, len(dialog)):

            if dialog[i][0] in speakers and dialog[i-1][0] in speakers: 
                name  = 'оба спикера'
                content = dialog[i-1][1] + dialog[i][1]
                dialog[i-1] = [] # заменим пустым списком, чтобы не изменять длину диалога
                dialog[i] = [name, content]
            
    # убрать пустые элементы
    dialog[:] = [i for i in dialog if len(i) > 0]       
    
    # оставить только реплики без указания на то, кто говорит
    dialog[:] = [i[1] for i in dialog]
    
    dialog[:] = list(map(lambda x: x.replace('(Микрофон отключён.)', ''), dialog))
    
    legit_dialog = get_legit_answer_question_structure(dialog)
    
    return legit_dialog

In [103]:
def get_all_clean_dialogues_by_commitee(session_dialog_commitee, session_stenograms_links_dict):
    
    '''
    На вход: переменная со списком, указывающим на даты заседения и
    для имеющих обсуждение пунктов повестки
    
    На выход: список с последовательными сессиями вопросов и ответов
    для определенного коммитета
    '''
    session_all_dialogs = []

    for row_chron in session_dialog_commitee.values.tolist():
        chron_date = row_chron[0]
        chron_date = chron_date[chron_date.find(' ', chron_date.find('заседания')) +1:]
        question_number = row_chron[1]
        indication = row_chron[2]
        chron_date = chron_date.replace('.', '')
    
        link_stenogram = session_stenograms_links_dict[chron_date]
        #print(link_stenogram, int(question_number), indication)

        certain_discussion = get_certain_discussion(link_stenogram, int(question_number), indication)
        if len(certain_discussion) == 0:
            continue
            
        # на сайте госдумы не везде правильная разметка, какие-то речи не выделены шрифтом
        # поэтому такие случаи отловим вручную
        if len(certain_discussion) % 2 == 0 and len(certain_discussion) != 0 :
            #session_all_dialogs.extend(certain_discussion) # старый вариант без дат
            discussion_with_date = [[chron_date],certain_discussion, [link_stenogram]]
            session_all_dialogs.append(discussion_with_date)
            
        for i in session_all_dialogs:
            if session_all_dialogs.count(i) > 1:
                session_all_dialogs.remove(i)
            
    return session_all_dialogs

In [96]:
def get_certain_discussion_new(url, number, additional_speech):
    number = str(number)
    #url = "http://transcript.duma.gov.ru/node/5735/"
    corpus = get_clean_transcript(url)

    markers = [number + '-й вопрос',
        'пункт ' + number,
        number + '-й пункт',
        'рассмотрению вопроса ' + number,
        number + '-й, проект',
        number + '-го вопроса',
        number + ' вопроса',
        number + '-му вопросу',
        'вопрос ' + number,
        number + ' вопрос',
        'пункту ' + number ]


    number_mentions = []

    # найти начало обсуждения определенного пункта
    for index, value in enumerate(corpus):
        
        speech_content = (value[1]).lower()

        # по-хорошему только Председательствующий объявляет пункт повестки
        if any(marker in speech_content for marker in markers)\
            and value[0] == 'Председательствующий.' and 'снять' not in speech_content: 
            
            # хотим избежать проблемы по типу 'вопрос 6' найден в 'вопрос 36'       
            seen_marker = [i for i in markers if i in speech_content][0]
            start_marker_location = speech_content.find(seen_marker)
            
            if speech_content[start_marker_location].isalpha(): # 1-й символ: 'вопрос 6'
                next_symbol = speech_content[start_marker_location + len(seen_marker)]
                
                if not next_symbol.isdigit():
                    number_mentions.append(index)
                
            # хотим избежать проблемы по типу '7 вопрос' найден в '27 вопрос'    
            if not speech_content[start_marker_location].isalpha():  # '7 вопрос'
                previous_symbol = speech_content[start_marker_location-1]
                
                if not previous_symbol.isdigit():
                    number_mentions.append(index) 
        
        # случай просьбы рассмотреть законопроект вне очереди
        if any(marker in speech_content for marker in markers) \
            and ('просьб' in value[1] or 'отклон' in speech_content)\
            and value[0] != 'Председательствующий.'\
            and 'снять'not in speech_content:
            
            # хотим избежать проблемы по типу 'вопрос 6' найден в 'вопрос 36'       
            seen_marker = [i for i in markers if i in speech_content][0]
            start_marker_location = speech_content.find(seen_marker)
            
            if speech_content[start_marker_location].isalpha(): # 1-й символ: 'вопрос 6'
                next_symbol = speech_content[start_marker_location + len(seen_marker)]
                
                if not next_symbol.isdigit():
                    number_mentions.append(index+1)
                
            # хотим избежать проблемы по типу '7 вопрос' найден в '27 вопрос'    
            if not speech_content[start_marker_location].isalpha():  # '7 вопрос'
                previous_symbol = speech_content[start_marker_location-1]
                
                if not previous_symbol.isdigit():
                    number_mentions.append(index+1)


    if len(number_mentions) > 0:
        start = number_mentions[0]
    if len(number_mentions) == 0: # ведущий не говорил четко о начале конкретного пункта
        return []

        # найти конец обсуждения
    for i in range(start+1, len(corpus)):
        if 'желающие выступить' in corpus[i][1].lower():
            end = i
            break
        if "результаты голосования" in corpus[i][1].lower():
            end = i
            break
        if "покажите результаты" in corpus[i][1].lower():
            end = i
            break

    # вырезать диалог без дополнительных комментариев
    dialog = corpus[start:end]
    dialog[:] = [i for i in dialog if i[0] != 'Председательствующий.']
    dialog[:] = [i for i in dialog if i[0] != 'Из зала.']

    # склеить реплики одного и того же человека (могли быть прерваны председателем)
    for i in range(1, len(dialog)):
        if dialog[i][0] == dialog[i-1][0]:
            name  = dialog[i][0]
            content = dialog[i-1][1] + dialog[i][1]
            dialog[i-1] = [] # заменим пустым списком, чтобы не изменять длину диалога
            dialog[i] = [name, content]

    # убрать пустые элементы
    dialog[:] = [i for i in dialog if len(i) > 0]


    person1 = dialog[0][0]
    if additional_speech == 1:
        person2 = dialog[1][0] 

        # убрать выступление докладчика и содокладчика, интересуют только вопросы и ответы

    dialog[:] = dialog[1:]
    if additional_speech == 1:   
        dialog[:] = dialog[1:]

        speakers = [person1, person2]

            # склеить ответы докладчика и содокладчика, если они рядом 
            # (отвечают на один и тот же вопрос)

        for i in range(1, len(dialog)):

            if dialog[i][0] in speakers and dialog[i-1][0] in speakers: 
                name  = 'оба спикера'
                content = dialog[i-1][1] + dialog[i][1]
                dialog[i-1] = [] # заменим пустым списком, чтобы не изменять длину диалога
                dialog[i] = [name, content]

        # убрать пустые элементы
    dialog[:] = [i for i in dialog if len(i) > 0]       

        # оставить только реплики без указания на то, кто говорит
    dialog[:] = [i[1] for i in dialog]
    dialog[:] = list(map(lambda x: x.replace('(Микрофон отключён.)', ''), dialog))
    
    legit_dialog = get_legit_answer_question_structure(dialog)
    
    return legit_dialog

# Сбор данных

In [97]:
# Ссылки на страницы с стенограмм всех сессий 8 созыва:

session_8_stenograms_pages = [
'http://transcript.duma.gov.ru/search/?sessid=5687&doctype=3&dt_start=&dt_end=&phrase1=',\
'http://transcript.duma.gov.ru/search/?by=date&sessid=5767&doctype=3&dt_start=&dt_end=&phrase1=&PAGEN_1=2',\
'http://transcript.duma.gov.ru/search/?sessid=5767&doctype=3&dt_start=&dt_end=&phrase1=',\
'http://transcript.duma.gov.ru/search/?sessid=5888&doctype=3&dt_start=&dt_end=&phrase1=',\
'http://transcript.duma.gov.ru/search/?sessid=5983&doctype=3&dt_start=&dt_end=&phrase1=']

# Ссылки на страницы с хрониками всех сессий 8 созыва:

session_8_chronicles_pages = \
['http://transcript.duma.gov.ru/search/?sessid=5687&doctype=2&dt_start=&dt_end=&phrase1=',\
'http://transcript.duma.gov.ru/search/?sessid=5767&doctype=2&dt_start=&dt_end=&phrase1=',\
'http://transcript.duma.gov.ru/search/?sessid=5888&doctype=2&dt_start=&dt_end=&phrase1=',\
'http://transcript.duma.gov.ru/search/?sessid=5983&doctype=2&dt_start=&dt_end=&phrase1=']

In [98]:
session_8_chronicles_links = get_all_chronicles_links(session_8_chronicles_pages)

In [99]:
session_8_stenograms_links_dict = get_all_stenograms_links(session_8_stenograms_pages)

In [100]:
session_8_discussed_numbers = get_all_discussed_numbers(session_8_chronicles_links)

In [101]:
session_8_dialogies_by_commities = \
    get_variables_links_by_commities(session_8_discussed_numbers, 8)

session_8_dialogies_by_commities

['session_8_dialog_commitee_trans',
 'session_8_dialog_commitee_kult',
 'session_8_dialog_commitee_semi',
 'session_8_dialog_commitee_agrar',
 'session_8_dialog_commitee_bezop',
 'session_8_dialog_commitee_other',
 'session_8_dialog_commitee_sobst',
 'session_8_dialog_commitee_ekono',
 'session_8_dialog_commitee_stroi',
 'session_8_dialog_commitee_ohran',
 'session_8_dialog_commitee_kontr',
 'session_8_dialog_commitee_grazh',
 'session_8_dialog_commitee_gosud',
 'session_8_dialog_commitee_finan',
 'session_8_dialog_commitee_bjudz',
 'session_8_dialog_commitee_obraz',
 'session_8_dialog_commitee_delam',
 'session_8_dialog_commitee_infor',
 'session_8_dialog_commitee_oboro',
 'session_8_dialog_commitee_zasch',
 'session_8_dialog_commitee_regio',
 'session_8_dialog_commitee_prosv',
 'session_8_dialog_commitee_zero',
 'session_8_dialog_commitee_promy',
 'session_8_dialog_commitee_fizic',
 'session_8_dialog_commitee_energ',
 'session_8_dialog_commitee_trudu',
 'session_8_dialog_commitee_pro

In [104]:
session_8_ready_dialogs = []
for comm in session_8_dialogies_by_commities:
    indication_comm = comm[len(comm) - comm[::-1].find('_'):]
    comm = eval(comm)
    variable_name = 'session_8_all_dialogs_'+ indication_comm
    print(variable_name)
    session_8_ready_dialogs.append(variable_name)
    
    # создание самой переменной
    globals()[variable_name] = get_all_clean_dialogues_by_commitee(\
       comm, session_8_stenograms_links_dict)

session_8_all_dialogs_trans
session_8_all_dialogs_kult
session_8_all_dialogs_semi
session_8_all_dialogs_agrar
session_8_all_dialogs_bezop
session_8_all_dialogs_other
session_8_all_dialogs_sobst
session_8_all_dialogs_ekono
session_8_all_dialogs_stroi
session_8_all_dialogs_ohran
session_8_all_dialogs_kontr
session_8_all_dialogs_grazh
session_8_all_dialogs_gosud
session_8_all_dialogs_finan
session_8_all_dialogs_bjudz
session_8_all_dialogs_obraz
session_8_all_dialogs_delam
session_8_all_dialogs_infor
session_8_all_dialogs_oboro
session_8_all_dialogs_zasch
session_8_all_dialogs_regio
session_8_all_dialogs_prosv
session_8_all_dialogs_zero
session_8_all_dialogs_promy
session_8_all_dialogs_fizic
session_8_all_dialogs_energ
session_8_all_dialogs_trudu
session_8_all_dialogs_promy
session_8_all_dialogs_ekolo
session_8_all_dialogs_mezhd


In [105]:
session_8_all_dialogs_ekono

[[['10 ноября 2022 г'],
  ['Ну, безусловно, положительное социальное значение данный законопроект имеет, и с гуманистической точки зрения тоже. Единственный вопрос возник у меня. Ну, тут упор делается на отсутствие оплаты, а если у человека и документов нет? Вот он может оставить дома документы или свидетельство о том, что он инвалид. Предусмотрен какой-то механизм или нужно это добавить? Ведь формально-то будет другое основание.',
   ' Я уверен, что люди с инвалидностью не будут злоупотреблять своим правом. И давайте будем относиться позитивно к этой норме, а не так, что с принятием этого закона возникнет очень много случаев злоупотребления своим правом.',
   'Уважаемый Михаил Борисович, важный законопроект, его актуальность сегодня лежит и в другой плоскости. В некоторых регионах по-прежнему остро стоит вопрос не о постыдной высадке, а в принципе о возможности посадки в общественный транспорт. Данный вопрос становится сверхважным в связи с проведением специальной военной операции. Мн

In [106]:
# # выгрузка готовых диалогов в формате txt

# for ready_dialog in session_8_ready_dialogs:
#     dialog = eval(ready_dialog)
#     with open(ready_dialog +'.txt', 'w', encoding='utf-8') as f:
#         for item in dialog:
#             f.write("%s\n" % item)
        

In [107]:
import pickle

In [114]:
# # выгрузка готовых диалогов в формате pickle

# for ready_dialog in session_8_ready_dialogs:
#     dialog = eval(ready_dialog)
#     with open(ready_dialog +'.pkl', 'wb') as file:
#         pickle.dump(dialog, file)

# Детекция ошибок

In [109]:
session_8_stenograms_links_dict['04 октября 2022 г']

'http://transcript.duma.gov.ru/node/5915/'

In [112]:
session_8_all_dialogs_kontr

[[['10 ноября 2022 г'],
  [' Уважаемый Андрей Викторович, мы хорошо помним, как в рамках так называемой гильотины, скажем, Минтранс предлагал срезать социальные гарантии для людей с инвалидностью. От частного к общему — мой вопрос: не приведёт ли передача полномочий к тому, что социальные гарантии для людей будут снижены, когда, например, Минтруд или какие-то другие министерства, которые заинтересованы в их сохранении, не будут принимать участие в решении данных вопросов?',
   ' Спасибо за вопрос. Я разделяю вашу озабоченность, её неоднократно высказывали наши ведомства, которые выступали согласовантами (такое неудачное слово) — участвовали в процедурах согласования данного законопроекта. Но вы знаете, по всем 56 сюжетам, которые охватывает этот законопроект... Напомню, мы сегодня всё-таки обсуждаем концепцию этого законопроекта, а она очевидна: есть необходимость упростить реализацию многих нормативных функций исполнительных органов государственной власти, спустив регулирование с уров